In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Physical_Exercise_Response/GSE142426'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Impact of short and long-term electrically induced muscle exercise on gene signaling pathways, gene expression, and PGC1a methylation in men with spinal cord injury"
!Series_summary	"Exercise attenuates the development of chronic non-communicable diseases (NCDs). Gene signaling pathway analysis offers an opportunity to discover if electrically induced muscle exercise regulates key pathways among people living with spinal cord injury (SCI). We examined short-term and long-term durations of electrically induced skeletal muscle exercise on complex gene signaling pathways, specific gene regulation, and epigenetic tagging of PGC1a, a major transcription factor in skeletal muscle of men with SCI. After short or long-term electrically induced exercise training, participants underwent biopsies of the trained and untrained muscles. RNA was hybridized to an exon microarray and analyzed using a gene set enrichment analysis. We discovered that long-term exerc

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # The data description suggests RNA hybridization to an exon microarray

# Determine the data availability
trait_row = 2  # intervention: EXP, intervention: CTL
age_row = None  # No explicit age data found
gender_row = 1  # Sex: male

# Define conversion functions
def convert_trait(value):
    value = value.split(":")[1].strip()
    return 1 if value == "EXP" else 0

def convert_age(value):
    return None  # Age data is not available

def convert_gender(value):
    value = value.split(":")[1].strip()
    return 1 if value == "male" else 0

# Save Metadata
save_cohort_info('GSE142426', './preprocessed/Physical_Exercise_Response/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Physical_Exercise_Response', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Physical_Exercise_Response/trait_data/GSE142426.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM4227228': [1, 1], 'GSM4227229': [1, 1], 'GSM4227230': [1, 1], 'GSM4227231': [1, 1], 'GSM4227232': [1, 1], 'GSM4227233': [1, 1], 'GSM4227234': [1, 1], 'GSM4227235': [1, 1], 'GSM4227236': [1, 1], 'GSM4227237': [1, 1], 'GSM4227238': [1, 1], 'GSM4227239': [1, 1], 'GSM4227240': [1, 1], 'GSM4227241': [1, 1], 'GSM4227242': [1, 1], 'GSM4227243': [0, 1], 'GSM4227244': [0, 1], 'GSM4227245': [0, 1], 'GSM4227246': [0, 1], 'GSM4227247': [0, 1], 'GSM4227248': [0, 1], 'GSM4227249': [0, 1], 'GSM4227250': [0, 1], 'GSM4227251': [0, 1], 'GSM4227252': [0, 1], 'GSM4227253': [0, 1], 'GSM4227254': [0, 1], 'GSM4227255': [0, 1], 'GSM4227256': [0, 1], 'GSM4227257': [0, 1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the key names for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
